# Введение в Pytorch

Авторы: Гирдюк Дмитрий, Никольская Анастасия

In [1]:
import numpy as np
import pandas as pd
import torch

## Тензоры

### Инициализация

In [ ]:
data = [[1, 2], [3, 4]]
tensor_data = torch.tensor(data)  # infer types
print(tensor_data, tensor_data.dtype, sep="\n")

In [ ]:
data_np = np.array(data)  # , dtype=np.int64
tensor_np = torch.from_numpy(data_np)
print(tensor_np, tensor_np.dtype, data_np.dtype, sep="\n")

In [ ]:
tensor_ones = torch.ones_like(tensor_np) # retains the properties of x_data
print(tensor_ones, tensor_ones.dtype)

tensor_rand = torch.rand_like(tensor_np, dtype=torch.float)  # remove dtype
print(tensor_rand, tensor_rand.dtype)

In [ ]:
shape = (2, 3)
tensor_rand = torch.rand(shape)
tensor_ones = torch.ones(shape)
tensor_zeros = torch.zeros(shape)

print(tensor_rand, tensor_ones, tensor_zeros, sep="\n")

In [ ]:
tensor_ones.to(torch.int16)

Типы данных: https://pytorch.org/docs/stable/tensors.html#data-types

Документация по способам создания тензоров: https://pytorch.org/docs/stable/torch.html#creation-ops

### Основные аттрибуты

In [ ]:
print(tensor_ones.shape, tensor_ones.dtype, tensor_ones.device)

### GPU!

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"  # "cuda:0"
tensor_ones.to(device)

### Индексация, слайсы, етц.

In [ ]:
tensor = torch.rand(3, 5)
tensor, tensor.shape

In [ ]:
tensor[0]

In [ ]:
tensor[:, 0]

In [ ]:
tensor[:2, :3]

In [ ]:
tensor.reshape(-1, 3)

In [ ]:
tensor.reshape(-1, 3).reshape(3, 5)

In [ ]:
tensor_unsqueezed = tensor.unsqueeze(-1)
tensor_unsqueezed, tensor_unsqueezed.shape

In [ ]:
tensor_squeezed = tensor_unsqueezed.squeeze(-1)
tensor_squeezed, tensor_squeezed.shape

### Основные операции

Операций больше сотни. Все работают и на GPU. Подробный список в документации: https://pytorch.org/docs/stable/torch.html

Если помните, как работать с numpy-массивами, проблем с адаптацией быть не должно.

In [ ]:
tensor_cat = torch.cat([tensor_rand, tensor_ones, tensor_zeros], dim=1)
tensor_cat

In [ ]:
tensor_rand + tensor_ones

In [ ]:
tensor_rand * tensor_zeros

In [ ]:
tensor_rand @ tensor_ones.T

In [ ]:
torch.matmul(tensor_rand, tensor_ones.T)

In [ ]:
tensor_rand.sum(), tensor_rand.sum().item()

In [ ]:
tensor_rand.add_(1)  # inplace

In [ ]:
tensor_rand.numpy()

Тензоры в массивах на CPU и numpy-массивы связаны, указывают на те же самые ячейки в памяти. Изменение одного приведет к изменению другого, и наоборот.

In [ ]:
tensor_rand_np = tensor_rand.numpy()
tensor_rand_np += 1.0
tensor_rand_np, tensor_rand

## Dataset и Dataloader

PyTorch предоставляет две абстракции для данных: torch.utils.data.Dataset и torch.utils.data.DataLoader. В Dataset хранятся сами данные (фичи, изображения, текст и др.) и значения целевой переменной/метки/таргеты/другие изображения/текст и т.д. А DataLoader оборачивает набор данных в так называемый iterable (русский аналог?), и позволяет итерироваться по датасету.

В торче и связанных с ним библиотеках хватает готовых датасетов. Рассмотрим датасет и даталоадер и использованием FashionMNIST.

In [ ]:
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
train_dataset = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_dataset = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}

In [ ]:
def show_fashion_plots(
    dataset: Dataset, labels_map: dict[int, str], cols: int = 3, rows: int = 3
) -> None:
    figure = plt.figure(figsize=(8, 8))
    for i in range(1, cols * rows + 1):
        sample_idx = torch.randint(len(dataset), size=(1,)).item()
        img, label = dataset[sample_idx]
        figure.add_subplot(rows, cols, i)
        if type(label) == torch.Tensor:
            plt.title(labels_map[label.item()])
        else:
            plt.title(labels_map[label])
        plt.axis("off")
        plt.imshow(img.squeeze(), cmap="gray")

    plt.show()

In [ ]:
show_fashion_plots(train_dataset, labels_map=labels_map)

Хорошо, как нам реализовать датасет для собственных нужд? Наследуемся от Dataset (или его потомков), и реализуем методы \_\_len\_\_ и \_\_getitem\_\_.

In [ ]:
from os.path import join as pjoin
from typing import Callable

from torchvision.datasets.mnist import read_image_file, read_label_file
from torchvision.io import read_image


class CustomFashionMNISTDataset(Dataset):
    def __init__(
        self,
        root: str,
        train: bool = True,
        transform: Callable | None = None,
        target_transform: Callable | None = None,
    ) -> None:
        self.train = train
        self.transform = transform
        self.target_transform = target_transform

        images_file = f"{'train' if self.train else 't10k'}-images-idx3-ubyte"
        labels_file = f"{'train' if self.train else 't10k'}-labels-idx1-ubyte"

        self._data = read_image_file(pjoin(root, images_file))
        self._targets = read_label_file(pjoin(root, labels_file))

    def __len__(self) -> int:
        return len(self._data)

    def __getitem__(self, idx) -> tuple[torch.Tensor, int]:
        image = self._data[idx]
        label = self._targets[idx]

        if self.transform:
            image = self.transform(image)

        if self.target_transform:
            label = self.target_transform(label)

        return image, label

In [ ]:
custom_train_dataset = CustomFashionMNISTDataset(root=pjoin("data", "FashionMNIST", "raw"), train=True)
custom_test_dataset = CustomFashionMNISTDataset(root=pjoin("data", "FashionMNIST", "raw"), train=False)

In [ ]:
show_fashion_plots(custom_train_dataset, labels_map=labels_map)

Наконец, оборачиваем датасет даталоадером

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=64, num_workers=0, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, num_workers=0, shuffle=True)

In [ ]:
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.title(f"Label: {label}")
plt.show()

Подробнее про модуль torch.utils.data в дкоументации: https://pytorch.org/docs/stable/data.html

## Module

Нейронные сети состоят из слоев/модулей, которые выполняют операции с данными. Пространство имен torch.nn предоставляет все строительные блоки, необходимые для создания собственной нейронной сети. Каждый модуль в pytorch является подклассом nn.Module. Нейронная сеть — это сам модуль, состоящий из других модулей (слоев).

In [ ]:
from torch import nn
from torchvision import datasets, transforms

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.flatten = nn.Flatten()  # векторизация изображения-матрицы
        self.linear_relu_stack = nn.Sequential(  # контейнер для модулей
            nn.Linear(  # линейная трансформация, bias=True on default
                in_features=28 * 28,
                out_features=512,
                bias=True
            ),
            nn.ReLU(),  # нелинейная функция активации
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(  # основной метод, связывающий инициализированные слои в вычислительный граф
        self, x: torch.LongTensor
    ) -> torch.FloatTensor:
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
model = NeuralNetwork().to(device)
print(model)

In [ ]:
logits = model(train_features.to(device))
pred_probas = nn.Softmax(dim=1)(logits)  # софтмакс, было ведь, ну было в прошлом семестре!
y_pred = pred_probas.argmax(1)
y_pred

In [ ]:
for name, param in model.named_parameters():
    print(f"Layer: {name}", f"Size: {param.size()}", f"Values : {param[:2]}", sep="\n", end="\n\n")

## Автоматическое дифференцирование

Для вычисления градиентов в pytorch реализован движок для автоматического дифференцирования вычислительных графов.

In [ ]:
x = torch.ones(5)
y = torch.zeros(3)
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [ ]:
print(z)
print(loss)

Пример выше -- однослойная нейронная сеть. Параметры _w_ и _b_ можно соптимизировать на основе имеющейся выборки. Обычно у нас на выходе есть некоторая лосс-функция, численно выражающая то, насколько хорошо наша модель справляется с поставленной задачей. Ее градиент по параметрам позволяет произвести эту оптимизацию.

Когда мы производим всевозможные манипуляции с тензорами (forward), за фасадом создается вычислительный граф, на основе которого могут быть вычислены производные. Получаем значение лосс-функции -> последовательно вычисляем ее градиент по параметрам в графе (backward).

In [ ]:
print(w.grad)
print(b.grad)

In [ ]:
loss.backward()

In [ ]:
print(w.grad)
print(b.grad)

А что для промежуточных вершин графа?

In [ ]:
z.grad

По умолчанию, все тензоры хранят свою вычислительную "историю" и поддерживают вычисление градиентов. Но это поведение часто необязательно. Например, когда нам нужны константные параметры, или когда параметры модели соптимизированы, и нам интересен лишь прямой (forward) проход.

In [ ]:
z = torch.matmul(x, w) + b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w) + b
print(z.requires_grad)

z = torch.matmul(x, w) + b
z_det = z.detach()
print(z_det.requires_grad)

DAGи (i want to apologize) в pytorch динамические. Каждый раз после вызова метода backward, autograd воссоздает DAG с нуля. Это именно то, что позволяет вставлять все эти ifы и циклы в моделях! При необходимости можно изменять shape, size и используемые операции на каждой итерации.

Подробное описание в документации: https://pytorch.org/docs/stable/notes/autograd.html